In [106]:
import jieba
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

In [107]:
# 确定文件路径
file_path="../data/cnews/cnews.test.txt"

In [108]:
with open(file_path,'r',encoding='utf-8',errors='ignore')as fp:
    lines=fp.readlines()

# 根据我们查看的文本的特性。取test中的3000样本
lines = lines[500:1000]+lines[1500:2000]+lines[2500:3000]+lines[3500:4000]+lines[4500:5000]+lines[5500:6000]
lines[0]

'体育\t华府收官战阿联送大礼 再获霸道数据不枉老板惊呼新浪体育讯华盛顿奇才在主场历经一个加时赛以95-94险胜强敌波士顿凯尔特人，这也是他们本赛季主场比赛的落幕战。易建联上场27分钟，8投3中得到6分10篮板2助攻1抢断，是球队篮板王。纵观整场比赛，作为球队的主要替补球员，易建联在这场双方命中率都不高的比赛中，进攻端发挥只能算是一般，虽然他在下半场进行了有效的调整，也取得了明显的改进，但从整体着眼，这种层次的进攻发挥只能算是中规中矩。不过，阿联在本场比赛中依然不乏亮点——他全场比赛抢下10个篮板球，其中3个是进攻篮板，两项数据均为全队最高。如果结合阿联27分19秒的出场时间，我们可以轻易发现，阿联的篮板效率依然是全队最高的——甚至也是全场最高的(杰梅因-奥尼尔与杰夫-格林的篮板球虽然超过了阿联，但出场时间更是远超阿联)。在本赛季的大部分时间里，阿联的篮板球能力都不那么突出。由于出场时间大幅削减的缘故，阿联本赛季场均只有3.7篮板进账，这一成绩位居奇才第8。就算计算36分钟理论数据，阿联也只以7.7个位居第6。如果从篮板争抢率上看，阿联的前场篮板率只有7.5%，位居球队第8，后场篮板率为17.2，位居球队第4，总篮板率为12.2，位居球队第6——无论从哪个角度看，阿联在本赛季的篮板球争抢上，都没什么优势。但阿联用行动证明，数据也是会骗人的——事实上，由于出场时间较少，希尔顿-阿姆斯特朗、凯文-塞拉芬等人的相关数据排在阿联之前，而这显然是不符合实际的。同时，阿联也在最近的比赛中证明了自己——只要给他足够的表现空间，阿联绝对能胜任球队主力篮板手的角色：4月以来，阿联在7场比赛中，有4场都扮演了球队替补篮板王的角色，而在上一场比赛中，虽然阿联没有成为球队替补篮板王，但他还是抢下了9个篮板，其中6个是进攻篮板，阿联的大爆发也让老板特德-利昂西斯直呼“阿联变身篮板野兽”。阿联确实没有让老板白白惊呼，本场比赛他再度展现了出色的篮板球意识。虽然在第一节没有获得出场时间，但他在余下三节狂揽篮板球：第二节单节抢下4个篮板，其中2个是进攻篮板，第三节再度抓下3个篮板，第四节则抢下3个篮板。三节比赛，不满26分钟，阿联就抢下了常规时间内全队最多的10个篮板，这份成绩已经足以告慰老板利昂西斯的那声惊叹了。不过阿联虽然如此勇猛，却没有得到主帅菲利普-桑德斯全方位的信任。在关键的加时赛中，阿联

In [109]:
# 考虑到文本的特殊性，我们先将取出来的文本特征先shuffle
random.shuffle(lines)
lines[0]

'家居\t家装捞过界改写市场格局8月8日，国内橱柜行业老大欧派橱柜将正式宣布进军广州家装市场。据悉，新成立的欧派铂金装饰公司，将全面涉足普通家装、精品家装及整体精装修业务，成为继博洛尼之后另一家朝产业链下游延伸的橱柜企业。与此同时，像靓家居、好美家等知名建材超市也在迅速渗透到家装行业，打出了各种按平方米计价的“装修套餐”，这项业务今年以来在广州市场上推行得风风火火，吸引不少计划装修的业主，外地建材卖场也在纷纷效仿。此外、一些实力雄厚的房地产商也在发展精装修市场，加上家电、家具等企业陆续进入传统的硬装或新兴的软装行业，这些都加速了家装行业的发展。业内分析，今年可能是家装市场变局的分水岭，这些新入行者有望改写目前广州家装原有运作模式。图为 家装捞过界改写市场格局现状：市场不景气分羹者众多受去年美国金融危机和国内房地产业萎缩的影响，今年上半年广州家装市场表现得颇为萧条。据大名装饰总经理邱红平透露，与去年同期相比，整体业务量起码下降30%以上，接单数量也明显萎缩。尽管家装行业如此不景气，却仍然不能阻挡从产业链上蜂拥而来分羹者。近日，记者获悉，国内橱柜行业老大欧派橱柜8月8日将在广州推出独立的铂金装饰公司，从材料、工艺、管理和服务上打造全新的“标准工程体系”，成为继博洛尼之后另一家朝产业链下游延伸的橱柜企业。而此前不久，尚品宅配开始加大设计中心的投入，计划从全屋家具定制延伸到涵盖软装领域的整体家居定制。与此同时，像好美家、靓家居这类本土卖场，目前开始重视装潢中心的“拉单”能力，纷纷在此基础上推出含主材、辅材、人工费的“全包套餐”，在今年市场上也颇受追捧。通过这种采用“材料+人工+税费”的零利润家装方式，一方面为建材产品提供一个更有诱惑力的出口；另一方面在不增加支出情况下，又可以获得消费者的好感和口碑。据了解，在家装业发展相对成熟的北京市场，建材卖场“跨界”做家装的做法并非新鲜事，东方家园和百安居均有装潢中心，居然之家也成立了“乐屋家装”，均取得不俗的市场反应。这些依据不同资源优势涉足家装业务的企业，不可避免对目前广州家装市场造成分流。去年3月份，北京东易日盛斥资2亿打造的木作工厂在北京通州区建成，自己生产家居装饰所需要的所有木制产品系列，从木门到衣柜乃至墙饰、全屋家具都一应俱全。在广州，星艺装饰、名匠装饰等为应对这种挑战，目前都纷纷加快在上游材料工厂的整合，同时也加大软装配饰

In [110]:
# 将文本中的label和text分开
label,text_x=list(),list()
for line in lines:
    line=line.split('\t')
    label.append(line[0])
    text_x.append(line[1])
label[:5]

['家居', '体育', '教育', '体育', '教育']

In [112]:
# 分词
text_x=[[each0 for each0 in jieba.cut(each)]for each in text_x]
# 将分词后的内容连接成list
text_x=[' '.join(each) for each in text_x]

In [113]:
text_x[0]

'家装   捞   过界   改写   市场   格局   8   月   8   日   ，   国内   橱柜   行业   老大   欧派   橱柜   将   正式   宣布   进军   广州   家装   市场   。   据悉   ，   新   成立   的   欧派   铂金   装饰   公司   ，   将   全面   涉足   普通   家装   、   精品   家装   及   整体   精装修   业务   ，   成为   继   博洛尼   之后   另一家   朝   产业链   下游   延伸   的   橱柜   企业   。   与此同时   ，   像   靓   家居   、   好美家   等   知名   建材超市   也   在   迅速   渗透到   家装   行业   ，   打出   了   各种   按   平方米   计价   的   “   装修   套餐   ”   ，   这项   业务   今年以来   在   广州   市场   上   推行   得   风风火火   ，   吸引   不少   计划   装修   的   业主   ，   外地   建材   卖场   也   在   纷纷   效仿   。   此外   、   一些   实力雄厚   的   房地产商   也   在   发展   精装修   市场   ，   加上   家电   、   家具   等   企业   陆续   进入   传统   的   硬装   或   新兴   的   软装   行业   ，   这些   都   加速   了   家装   行业   的   发展   。   业内   分析   ，   今年   可能   是   家装   市场   变局   的   分水岭   ，   这些   新入   行者   有望   改写   目前   广州   家装   原有   运作   模式   。   图为       家装   捞   过界   改写   市场   格局   现状   ：   市场   不景气   分羹者   众多   受   去年   美国   金融危机   和   国内   房地产业   萎缩   的   影响   ，   今年   上半年   广州   家装   市场   表

In [114]:
# 将数据分为训练集和测试集
train_X=text_x[:2500]
train_y=label[:2500]
test_X=text_x[2500:]
test_y=label[2500:]

In [117]:
count_vector = CountVectorizer()
# 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j]表示j词在i类文本下的词频
vector_matrix = count_vector.fit_transform(train_X)

# tfidf度量模型
train_tfidf = TfidfTransformer(use_idf=False).fit_transform(vector_matrix)

In [118]:
# 调用MultinomialNB分类器进行训练
clf = MultinomialNB().fit(train_tfidf,train_y)

In [119]:

# 测试
test_vector = count_vector.transform(test_X)
test_tfidf = TfidfTransformer(use_idf=False).fit_transform(test_vector)
predict_result = clf.predict(test_tfidf)

In [122]:

# 评测预测效果。直接使用准确率作为评判标准
def accuracy_(test_y,predict):
    TP,num = 0,len(test_y)
    for i in range(num):
        if test_y[i]==predict[i]:
            TP+=1
    return TP/num

In [123]:
print('多项式模型分类效果：%f'%accuracy_(test_y,predict_result))

多项式模型分类效果：0.970000


使用SVM进行分类
---

In [124]:
from sklearn import svm

In [125]:
clf=svm.LinearSVC(C=1.6,max_iter=2000,multi_class='ovr',penalty='l2',tol=0.0001,random_state=42)
clf.fit(train_tfidf,train_y)

LinearSVC(C=1.6, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=2000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [126]:
# 使用svm分类测试
predict_result=clf.predict(test_tfidf)

In [128]:
print('SVM模型分类效果：%f'%accuracy_(test_y,predict_result))

SVM模型分类效果：0.976000
